<a href="https://colab.research.google.com/github/2-sehee/ASAC_5th_Review_Summarization/blob/main/Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data/lib load

In [ ]:
import pandas as pd
import numpy as np
from transformers import pipeline
from transformers import AutoTokenizer
from tqdm import tqdm
import pickle
import os


import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#@title Encoding 완료된 리뷰(260만개)

## preprocessing 완료된 데이터를 불러옴

encoding_review = pd.read_csv('/content/drive/MyDrive/ASAC/기업연계_프로젝트/final_dataset/review_encoding.csv')
encoding_review.head(5)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,text_length
0,2348758,1575984,80739,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,7
1,3273772,1231666,98537,4.0,1,0,1,Cute interior and owner gave us tour of upco...,2017-01-14 20:54:15,9
2,5406063,898460,94651,4.0,1,0,1,I was really between 3 and 4 stars for this on...,2018-07-17 03:30:07,20
3,44775,644900,92221,5.0,0,0,0,First time there and it was excellent!!! It fe...,2017-02-19 13:32:05,5
4,3891834,177785,92778,5.0,0,0,0,"Great burgers,fries and salad! Burgers have a...",2017-01-02 03:17:34,5


In [3]:
#@title 샘플링 + 문장 분리 완료 데이터(3만개)

sampling = pd.read_csv('/content/drive/MyDrive/ASAC/기업연계_프로젝트/final_dataset/sampling/sample_review.csv')
sampling.head(1)

,index,review_id,user_id,business_id,stars,useful,funny,cool,text,date,sentences,sentence_counts,keywords_ug,keywords_bg
0,184,1443659,607521,108373,5.0,0,0,0,Found 51st Deli on Uber Eats and we loved it!!...,2017-01-14 21:23:10,['Found 51st Deli on Uber Eats and we loved it...,2,"['uber', 'wait']","['deli uber', 'check person']"


In [4]:
#@title 문장 단위 vader, roberta 감성 분석 결과 데이터 (문장 약 23만개)
vader = pd.read_csv('/content/drive/MyDrive/ASAC/기업연계_프로젝트/sentiment_analysis_result/Vader_sentiment_analysis_result.csv')
roberta = pd.read_csv('/content/drive/MyDrive/ASAC/기업연계_프로젝트/sentiment_analysis_result/Roberta_sentiment_analysis_result.csv')

In [5]:
## vader 결과는 필요없는 컬럼은 드랍하지 않고 저장하였음.
vader.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,date,sentence,compound_score,positive_score,neutral_score,negative_score
0,1443659,607521,108373,5.0,0,0,0,2017-01-14 21:23:10,Found 51st Deli on Uber Eats and we loved it!!!,0.989771,0.991007,0.007758,0.001236


In [6]:
## roBERTa 데이터는 필요없는 정보는 버리고 최종 사용할 컬럼으로 유지.
roberta.head(1)

,review_id,stars,sentence,compound_score,positive_score,neutral_score,negative_score
0,1443659,5.0,Found 51st Deli on Uber Eats and we loved it!!!,0.989771,0.991007,0.007758,0.001236


In [7]:
#@title 최종 데이터셋 (결측치 처리 완료)

## 1. Review
review = pd.read_csv('/content/drive/MyDrive/ASAC/기업연계_프로젝트/final_dataset/finalDB/review.csv')

## 2. User
user = pd.read_csv('/content/drive/MyDrive/ASAC/기업연계_프로젝트/final_dataset/finalDB/user.csv')

## 3. Business
business = pd.read_csv('/content/drive/MyDrive/ASAC/기업연계_프로젝트/final_dataset/finalDB/business.csv')

## 4. Sentence -> Roberta 모델로 분류 완성한 데이터
sentence = pd.read_csv('/content/drive/MyDrive/ASAC/기업연계_프로젝트/final_dataset/sampling/Roberta_sentiment_add_topics.csv')

In [8]:
sentence.head()

,review_id,stars,sentence,compound_score,positive_score,neutral_score,negative_score,keywords_ug,keywords_bg,topic
0,1443659,5,Found 51st Deli on Uber Eats and we loved it!!!,0.989771,0.991007,0.007758,0.001236,uber,deli uber,service
1,1443659,5,I can't wait to check it out in person!,0.985175,0.986700,0.011776,0.001525,wait,check person,others
2,1044648,5,Love love love this place for HH n dinner is g...,0.989841,0.991794,0.006252,0.001954,dinner,dinner good,others
3,1044648,5,"Its ok for brunch, not as great as what people...",-0.041061,0.229077,0.500784,0.270139,brunch,brunch great,others
4,1044648,5,The Asian Nachos are the dish to get for HH.,0.397284,0.408822,0.579640,0.011538,nachos,asian nachos,food


In [13]:
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651 entries, 0 to 650
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   business_id         651 non-null    int64  
 1   name                651 non-null    object 
 2   address             644 non-null    object 
 3   category            651 non-null    object 
 4   region              651 non-null    object 
 5   review_count_biz    651 non-null    float64
 6   average_stars_biz   651 non-null    float64
 7   atmosphere_keyword  651 non-null    object 
 8   facility_keyword    651 non-null    object 
 9   food_keyword        651 non-null    object 
 10  others_keyword      651 non-null    object 
 11  price_keyword       651 non-null    object 
 12  service_keyword     651 non-null    object 
 13  service_score       651 non-null    float64
 14  others_score        651 non-null    float64
 15  food_score          651 non-null    float64
 16  price_sc

In [22]:
sampling.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30880 entries, 0 to 30879
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            30880 non-null  int64  
 1   review_id        30880 non-null  int64  
 2   user_id          30880 non-null  int64  
 3   business_id      30880 non-null  int64  
 4   stars            30880 non-null  float64
 5   useful           30880 non-null  int64  
 6   funny            30880 non-null  int64  
 7   cool             30880 non-null  int64  
 8   text             30880 non-null  object 
 9   date             30880 non-null  object 
 10  sentences        30880 non-null  object 
 11  sentence_counts  30880 non-null  int64  
 12  keywords_ug      30880 non-null  object 
 13  keywords_bg      30880 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 3.3+ MB


### 결측치 처리 방식

In [ ]:
## 감성 점수가 비어있다는 것은 특정 토픽에 해당하는 문장이 하나도 없었다는 것을 의미
## 따라서, 해당 내용을 0으로 채웠음.
business['facility_score'] = business['facility_score'].fillna(0)
business['service_score'] = business['service_score'].fillna(0)
business['others_score'] = business['others_score'].fillna(0)
business['food_score'] = business['food_score'].fillna(0)
business['price_score'] = business['price_score'].fillna(0)
business['atmosphere_score'] = business['atmosphere_score'].fillna(0)

In [ ]:
## 키워드 추출 결과 아무것도 없는 Nan 값이 있을 수 있는데, 이는 토픽에 해당하는 문장이 없어서
## 키워드를 추출하지 못한 것이므로 해당 결과를 문자열 none으로 채워주었음.

business['food_keyword']= business['food_keyword'].fillna('none')
business['others_keyword'] = business['others_keyword'].fillna('none')
business['atmosphere_keyword'] = business['atmosphere_keyword'].fillna('none')
business['facility_keyword'] = business['facility_keyword'].fillna('none')
business['service_keyword'] = business['service_keyword'].fillna('none')
business['price_keyword'] = business['price_keyword'].fillna('none')

In [25]:
#@title csv 불러온 경우 문자열 -> 리스트 or 딕셔너리 형으로 변환
# csv를 read_csv로 불러오면 숫자로 인식되는게 아니라 문자열로 인식함.
# 따라서, 아래의 처리를 해주어야 한다.
import pandas as pd
import ast
from ast import literal_eval

# compound_score을 float형으로 바꾸는 함수
def clean_and_parse_list(column):
    try:
        cleaned_column = column.strip() # 에러를 방지하기 위해 띄어쓰기가 있는 경우 이를 제거해줌
        return ast.literal_eval(cleaned_column) # 문자열을 리스트, 딕셔너리가 안에 포함된 경우 인식할 수 있게 바꿔줌
    except (ValueError, SyntaxError):
        return column  # 감성점수 결과를 결측치 처리전 전에는 nan도 있었으므로 그 경우에는 그대로 리턴

In [27]:
sampling

,index,review_id,user_id,business_id,stars,useful,funny,cool,text,date,sentences,sentence_counts,keywords_ug,keywords_bg
0,184,1443659,607521,108373,5.0,0,0,0,Found 51st Deli on Uber Eats and we loved it!!...,2017-01-14 21:23:10,['Found 51st Deli on Uber Eats and we loved it...,2,"['uber', 'wait']","['deli uber', 'check person']"
1,207,1044648,182108,134961,5.0,0,0,0,Love love love this place for HH n dinner is g...,2017-08-07 23:48:51,['Love love love this place for HH n dinner is...,11,"['dinner', 'brunch', 'nachos', 'entree', 'dish...","['dinner good', 'brunch great', 'asian nachos'..."
2,237,5693786,749220,54049,5.0,0,0,0,Not sure the significance behind the name and ...,2018-09-25 14:49:55,['Not sure the significance behind the name an...,4,"['restaurant', 'business', 'lunch', 'open']","['logo restaurant', 'business', 'menu breakfas..."
3,277,1166589,572170,9263,2.0,1,0,0,The service is going to be slooooww. And the f...,2017-10-02 01:52:47,"['The service is going to be slooooww.', 'And ...",7,"['slooooww', 'food', 'group', 'group', 'server...","['going slooooww', 'food okay', 'group slow', ..."
4,323,6678286,816928,108373,5.0,0,0,0,Much more than a deli! Wife and I had the Phil...,2017-12-29 20:18:59,"['Much more than a deli!', 'Wife and I had the...",6,"['deli', 'philly', 'tasty', 'chips', 'hungry',...","['deli', 'philly club', 'tasty good', 'bags ch..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30875,2679757,5143901,1772582,22595,1.0,0,0,0,"Brand new but filthy. Trash overflowing, dirty...",2021-11-07 23:54:35,"['Brand new but filthy.', 'Trash overflowing, ...",5,"['brand', 'trash', 'cleaning', 'overcharged', ...","['brand new', 'trash overflowing', 'counter cl..."
30876,2679857,2367318,995712,133866,3.0,8,2,3,Stopped in for dinner and drinks on a Saturday...,2018-03-25 15:30:29,['Stopped in for dinner and drinks on a Saturd...,15,"['dinner', 'parking', 'smokers', 'seating', 's...","['stopped dinner', 'parking sparse', 'smokers ..."
30877,2679863,6085901,1283829,84658,5.0,1,0,2,"Came here for lunch with my girlfriend, place ...",2018-08-10 04:52:50,"['Came here for lunch with my girlfriend, plac...",4,"['pancakes', '', 'berry', 'lunch']","['pancakes', '', 'blue berry', 'fruity drink']"
30878,2679972,5916423,1410939,123182,1.0,0,0,0,Used to go here every week. Won't go again. Ma...,2020-06-03 00:49:05,"['Used to go here every week.', ""Won't go agai...",10,"['week', 'won', 'bag', 'food', 'huang', 'recei...","['used week', 'won', 'bag leave', 'didn food',..."


In [ ]:
# 실행 예시
df = sentence
df['compound_score'] = df['compound_score'].apply(clean_and_parse_list)
# Check a few entries to ensure correct parsing
df['compound_score'].head(10)

# 문자열로 된 리스트를 실제 리스트로 변환하는 함수
def parse_list_column(column):
    return column.apply(ast.literal_eval)

# 'sentences' 컬럼을 리스트로 변환
df['sentences'] = parse_list_column(df['sentences'])
df['topics'] = parse_list_column(df['topics'])

# 1. 감성분석 모델링

In [15]:
#@title roberta 모델 불러오기
from transformers import pipeline #허깅페이스의 transformers의 pipeline을 인풋시켜야 사용 가능

sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
#@title 사용 예시
b = sentiment_analysis('''Don't be stupid enough to eat here is. It is expensive and crappy.
Just eat at the cafeteria. My friends and myself came to humpty's to have a nice breakfast and it was horrid and a disaster.
the service was fine, the food was DESPICABLE.
My friend ordered the chicken louie the mashed potatoes tasted like acid reflux and the chicken was so rubbery she couldn't eat half of it. The ground beef in the Mexican scrambler was questionable too.
My other friend ordered the crunchy chicken parmesan the pasta was watery and definitely unappealing and the vegetables were cold and clearly frozen. we all feel sick.''')

print(b[0]['label'])
print(b[0]['score'])

In [16]:
#@title 전체 실행 함수 코드
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import ast
from tqdm import tqdm
import torch

def load_roberta_model(model_name="cardiffnlp/twitter-roberta-base-sentiment"):
    """
    RoBERTa 모델과 토크나이저를 로드합니다.

    Args:
        model_name (str): 사용할 모델의 이름. 기본값은 'cardiffnlp/twitter-roberta-base-sentiment'.

    Returns:
        tokenizer: 로드된 토크나이저.
        model: 로드된 RoBERTa 모델.
        device: 사용할 디바이스 (GPU 또는 CPU).
    """
    print("모델과 토크나이저 로드 중...")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    print(f"모델과 토크나이저 로드 완료 (디바이스: {device})")
    return tokenizer, model, device

def polarity_scores_roberta(text, tokenizer, model, device):
    """
    RoBERTa 모델을 사용하여 텍스트의 감성 분석 점수를 반환합니다.

    Args:
        text (str): 감성 분석할 텍스트.
        tokenizer: 토크나이저.
        model: 감성 분석 모델.
        device: 사용할 디바이스 (GPU 또는 CPU).

    Returns:
        dict: 감성 점수 (positive, neutral, negative).
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    outputs = model(**inputs)
    scores = outputs.logits.detach().cpu().numpy()[0]  # GPU 텐서를 CPU로 이동
    scores = softmax(scores)
    return {
        "roberta_negative": scores[0],
        "roberta_neutral": scores[1],
        "roberta_positive": scores[2]
    }

def analyze_sentiments(df):
    """
    데이터프레임의 각 문장에 대해 RoBERTa 감성 분석을 수행
    Args:
        df: 분석할 데이터프레임, 'review_id', 'user_id', 'business_id', 'stars', 'date', 'sentences' 열이 필요함.
    Returns:
        results_df: 감성 분석 결과를 포함한 새로운 데이터프레임.
        생성된 데이터프레임은 문장단위로 만들어짐.
    """
    tokenizer, model, device = load_roberta_model()
    rows = []
    print("감성 분석 수행 중...")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        review_id = row['review_id']
        user_id = row['user_id']
        business_id = row['business_id']
        stars = row['stars']
        date = row['date']
        sentences_list = ast.literal_eval(row['sentences'])  # csv 불러올 때 문자열로 저장된 리스트를 리스트로 변환
        for sentence in sentences_list: #문장 단위로 실행
            scores = polarity_scores_roberta(sentence, tokenizer, model, device)
            compound_score = scores["roberta_positive"] - scores["roberta_negative"]
            rows.append([review_id, user_id, business_id, stars,date, sentence, compound_score, scores["roberta_positive"], scores["roberta_neutral"], scores["roberta_negative"]])

    print("문장 단위 감성 분석 완료")

    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame(rows, columns=['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'date', 'sentence', 'compound_score', 'positive_score', 'neutral_score', 'negative_score'])
    return result_df

### 실행
roberta = analyze_sentiments(sampling)

In [ ]:
#@title Roberta-base 평점별 박스 플롯 확인

plt.figure(figsize=(12, 8))
sns.boxplot(x='stars', y='compound_score', data=roberta, showfliers=False, linewidth=1.5, palette='Set3')
sns.stripplot(x='stars', y='compound_score', data=roberta, color='black', alpha=0.3, jitter=0.2, size=2)
plt.ylim(-1, 1)  # Y축을 -1부터 1까지 설정
plt.title('Roberta Sentiment Score Distribution by Star Rating', fontsize=16)
plt.xlabel('Star Rating', fontsize=14)
plt.ylabel('Compound Sentiment Score', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

In [ ]:
#@title 리뷰 데이터프레임에 평균값으로 대표하여 저장

import pandas as pd
from tqdm import tqdm

def filter_and_average_scores(scores):
    """
    주어진 점수 목록에서 NaN 값을 제거한 후, 나머지 점수들의 평균을 계산하여 반환.

    :param scores: 점수 목록 (시리즈 또는 리스트 형태)
    :return: 평균 점수 또는 None
    """
    if isinstance(scores, pd.Series):
        scores = scores.dropna().values.tolist()
    numeric_scores = [float(score) for score in scores if isinstance(score, (int, float))]
    if numeric_scores:
        return sum(numeric_scores) / len(numeric_scores)
    else:
        return None

def extract_sentiment_scores(df):
    """
    데이터프레임에서 각 비즈니스 별 주제에 따른 감성 점수를 추출한 후,
    결과를 새로운 데이터프레임에 추가.

    :param df: 입력 데이터프레임 (business_id, topic, compound_score 컬럼 필요)
    :return: 새로운 감성 점수가 포함된 데이터프레임
    """
    scores_by_business = {}

    for business_id in tqdm(df['business_id'].unique(), desc="Processing businesses"):
        business_scores = {}

        # 비즈니스별 고유한 모든 토픽 추출
        all_topics = df[df['business_id'] == business_id]['topic'].explode().unique()

        # 각 토픽에 대해 감성 점수 계산
        for topic in all_topics:
            compound_scores = df[
                (df['business_id'] == business_id) &
                (df['topic'].apply(lambda x: topic in x))
            ]['compound_score'].explode()

            average_score = filter_and_average_scores(compound_scores)

            if average_score is not None:
                business_scores[f'{topic}_score'] = average_score

        scores_by_business[business_id] = business_scores

    # 감성 점수를 데이터프레임으로 변환
    scores_df = pd.DataFrame.from_dict(scores_by_business, orient='index').reset_index()
    scores_df.rename(columns={'index': 'business_id'}, inplace=True)

    # 원본 데이터프레임과 감성 점수 데이터프레임을 병합
    df_with_scores = pd.merge(df, scores_df, on='business_id', how='left')

    return df_with_scores

# 함수 사용 예시
df_with_scores = extract_sentiment_scores(roberta)

# 2. 최종 키워드 추출(가게 + 토픽별)

In [20]:
#@title keybert 설치
!pip install keybert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.8 MB/s eta 0:00:00


In [22]:
#@title 문장별 단위 데이터에 가게 id추가
review_biz = review[['review_id','business_id']]
df_with_scores = df_with_scores.merge(review_biz, on='review_id')

In [23]:
df_with_scores.head(5)

,review_id,stars,sentence,compound_score,positive_score,neutral_score,negative_score,keywords_ug,keywords_bg,topic,business_id
0,1443659,5,Found 51st Deli on Uber Eats and we loved it!!!,0.989771,0.991007,0.007758,0.001236,uber,deli uber,service,108373
1,1443659,5,I can't wait to check it out in person!,0.985175,0.986700,0.011776,0.001525,wait,check person,others,108373
2,1044648,5,Love love love this place for HH n dinner is g...,0.989841,0.991794,0.006252,0.001954,dinner,dinner good,others,134961
3,1044648,5,"Its ok for brunch, not as great as what people...",-0.041061,0.229077,0.500784,0.270139,brunch,brunch great,others,134961
4,1044648,5,The Asian Nachos are the dish to get for HH.,0.397284,0.408822,0.579640,0.011538,nachos,asian nachos,food,134961


In [24]:
grouped = df_with_scores.groupby(['business_id', 'topic'])['sentence'].apply(lambda x: ' '.join(x)).reset_index()

In [25]:
grouped.head(5)

,business_id,topic,sentence
0,11,atmosphere,"First off we saw the ""open"" sign, but nobody w..."
1,11,facility,The place is always clean and the staff is sup...
2,11,food,I always pass the sign for sweet lizard on my ...
3,11,others,"To be honest I wasn't expecting much, but I wa..."
4,11,price,We both left with a greasy artificial thicknes...


In [ ]:
import pandas as pd
from keybert import KeyBERT
from tqdm import tqdm

# KeyBERT 초기화
kw_model = KeyBERT()

def extract_keywords(text, num_keywords=1):
    """
    주어진 텍스트에서 KeyBERT를 사용하여 키워드를 추출.

    Input
    1. text: 입력 텍스트
    2. num_keywords: 추출할 키워드의 수 (기본값: 1)

    Return: 추출된 키워드 리스트
    """
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=num_keywords)
    return [kw[0] for kw in keywords]

def apply_keyword_extraction(df, text_column='sentence', num_keywords=1):
    """
    데이터프레임의 텍스트 컬럼에 KeyBERT를 적용하여 키워드를 추출하고,
    새로운 'keywords' 컬럼을 추가.

    Input
    1. df -> 입력 데이터프레임
    2. text_column -> 텍스트가 포함된 컬럼 이름 (기본값: 'sentence')
    3. num_keywords -> 각 텍스트에서 추출할 키워드 수 (기본값: 1)

    Return: 키워드가 포함된 데이터프레임
    """
    # tqdm을 pandas에 적용하여 진행 상황 표시
    tqdm.pandas()

    # 키워드 추출 적용
    df['keywords'] = df[text_column].progress_apply(lambda text: extract_keywords(text, num_keywords))

    return df

# 함수 사용 예시
df_with_scores = apply_keyword_extraction(grouped, text_column='sentence', num_keywords=1)

In [ ]:
#@title 최종 데이터셋 완성
import pandas as pd

# Pivot the df_with_scores to organize the keywords by topic
pivoted_keywords = df_with_scores.pivot_table(index='business_id',
                                              columns='topic',
                                              values='keywords',
                                              aggfunc=lambda x: ' '.join(str(v) for v in x))

# Rename the columns to match the requested format
pivoted_keywords.columns = [f'{col}_keyword' for col in pivoted_keywords.columns]

# Select the score columns from df_with_scores
score_columns = ['business_id', 'service_score', 'others_score', 'food_score',
                 'price_score', 'atmosphere_score', 'facility_score']

# Merge the score columns and the pivoted keywords into a single dataframe
merged_scores_keywords = df_with_scores[score_columns].drop_duplicates().merge(
    pivoted_keywords, on='business_id', how='left')

# Merge this result with the business data
final_df = business.merge(merged_scores_keywords, on='business_id', how='left')

# Save the final dataframe to a CSV file
final_df.to_csv('final_business_data_with_scores_keywords.csv', index=False)